# Task
Run a pretrained decoder-only model and report its power consumption statistics.

## Setup environment

### Subtask:
Install necessary libraries for running the model and measuring power consumption.


**Reasoning**:
The first step is to install the `transformers` library and identify potential libraries for measuring power consumption. I will use `pip` for installation within a code block. For power consumption, I will install `pynvml` for NVIDIA GPUs as it's a common scenario in ML, and add comments about other potential tools for different systems.



In [1]:
%pip install transformers
# For NVIDIA GPUs, pynvml can be used to monitor GPU usage and power.
%pip install pynvml
# For CPU power consumption, tools like 'powertop' (Linux) or platform-specific libraries
# might be needed, but these often require system-level access or are OS-dependent.
# For this task, we will focus on GPU power consumption using pynvml as it's common
# for running decoder-only models.

## Load model

### Subtask:
Load a pretrained decoder-only model.


**Reasoning**:
Import the necessary classes from the transformers library and load the tokenizer and model for a pretrained decoder-only model.



In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

print(f"Tokenizer and model for '{model_name}' loaded successfully.")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Tokenizer and model for 'gpt2' loaded successfully.


## Prepare input

### Subtask:
Prepare sample input data for the model.


**Reasoning**:
Define sample text, encode it using the loaded tokenizer, and print both the original and encoded text.



In [3]:
# 1. Define a sample string of text
sample_text = "This is a sample input for the decoder-only model."

# 2. Use the loaded tokenizer to encode the sample text
# Ensure the output format is PyTorch tensors
encoded_input = tokenizer(sample_text, return_tensors="pt")

# 3. Print the original text and the encoded input
print("Original Text:", sample_text)
print("Encoded Input:", encoded_input)

Original Text: This is a sample input for the decoder-only model.
Encoded Input: {'input_ids': tensor([[ 1212,   318,   257,  6291,  5128,   329,   262,   875, 12342,    12,
          8807,  2746,    13]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


## Run inference and measure power

### Subtask:
Run the model on the input data while simultaneously measuring power consumption. This step might require specific tools or libraries depending on the hardware.


**Reasoning**:
The task requires measuring GPU power consumption before and after running the model. This involves using the `pynvml` library to get a handle to the GPU, query its power consumption, run the model, and then query the power consumption again. Finally, the difference is calculated and printed, and `pynvml` is shut down. All these steps can be grouped into a single code block.



In [4]:
import pynvml
import time

# 2. Initialize pynvml
try:
    pynvml.nvmlInit()
    print("pynvml initialized successfully.")
except pynvml.NVMLError as err:
    print(f"Error initializing pynvml: {err}")
    # Finish the task with failure if pynvml cannot be initialized
    raise

# 3. Get a handle to the first GPU device
try:
    handle = pynvml.nvmlDeviceGetHandleByIndex(0)
    print("GPU device handle obtained successfully.")
except pynvml.NVMLError as err:
    print(f"Error getting device handle: {err}")
    # Finish the task with failure if device handle cannot be obtained
    pynvml.nvmlShutdown()
    raise

# 4. Get the current power consumption of the GPU before running inference.
try:
    power_before = pynvml.nvmlDeviceGetPowerUsage(handle) / 1000.0  # Power in Watts
    print(f"GPU power consumption before inference: {power_before:.2f} W")
except pynvml.NVMLError as err:
    print(f"Error getting power usage before inference: {err}")
    # Finish the task with failure if power usage cannot be obtained
    pynvml.nvmlShutdown()
    raise

# 5. Run the loaded model with the prepared input data.
# Assuming 'model' and 'encoded_input' are already defined from previous steps
print("Running model inference...")
# Move model and input to GPU if available
if torch.cuda.is_available():
    model.to('cuda')
    encoded_input = {key: val.to('cuda') for key, val in encoded_input.items()}

with torch.no_grad(): # Inference should be done without gradient calculation
    outputs = model(**encoded_input)

print("Model inference completed.")

# Add a small delay to allow power consumption to stabilize after inference
time.sleep(1)

# 6. Get the current power consumption of the GPU after running inference.
try:
    power_after = pynvml.nvmlDeviceGetPowerUsage(handle) / 1000.0  # Power in Watts
    print(f"GPU power consumption after inference: {power_after:.2f} W")
except pynvml.NVMLError as err:
    print(f"Error getting power usage after inference: {err}")
    # Finish the task with failure if power usage cannot be obtained
    pynvml.nvmlShutdown()
    raise

# 7. Calculate the power consumed during inference (difference between after and before).
power_consumed_during_inference = power_after - power_before

# 8. Print the power consumption before and after inference, and the calculated consumption during inference.
print(f"\n--- Power Consumption Statistics ---")
print(f"Power consumption before inference: {power_before:.2f} W")
print(f"Power consumption after inference: {power_after:.2f} W")
print(f"Power consumed during inference: {power_consumed_during_inference:.2f} W")
print(f"-----------------------------------")


# 9. Shutdown pynvml.
pynvml.nvmlShutdown()
print("pynvml shutdown successfully.")

pynvml initialized successfully.
GPU device handle obtained successfully.
GPU power consumption before inference: 10.56 W
Running model inference...


NameError: name 'torch' is not defined

**Reasoning**:
The previous code block failed because the `torch` library was not imported. I need to import `torch` before attempting to use `torch.cuda.is_available()`. The rest of the logic for power measurement should remain the same.



In [5]:
import pynvml
import time
import torch

# 2. Initialize pynvml
try:
    pynvml.nvmlInit()
    print("pynvml initialized successfully.")
except pynvml.NVMLError as err:
    print(f"Error initializing pynvml: {err}")
    # Finish the task with failure if pynvml cannot be initialized
    raise

# 3. Get a handle to the first GPU device
try:
    handle = pynvml.nvmlDeviceGetHandleByIndex(0)
    print("GPU device handle obtained successfully.")
except pynvml.NVMLError as err:
    print(f"Error getting device handle: {err}")
    # Finish the task with failure if device handle cannot be obtained
    pynvml.nvmlShutdown()
    raise

# 4. Get the current power consumption of the GPU before running inference.
try:
    power_before = pynvml.nvmlDeviceGetPowerUsage(handle) / 1000.0  # Power in Watts
    print(f"GPU power consumption before inference: {power_before:.2f} W")
except pynvml.NVMLError as err:
    print(f"Error getting power usage before inference: {err}")
    # Finish the task with failure if power usage cannot be obtained
    pynvml.nvmlShutdown()
    raise

# 5. Run the loaded model with the prepared input data.
# Assuming 'model' and 'encoded_input' are already defined from previous steps
print("Running model inference...")
# Move model and input to GPU if available
if torch.cuda.is_available():
    model.to('cuda')
    encoded_input = {key: val.to('cuda') for key, val in encoded_input.items()}

with torch.no_grad(): # Inference should be done without gradient calculation
    outputs = model(**encoded_input)

print("Model inference completed.")

# Add a small delay to allow power consumption to stabilize after inference
time.sleep(1)

# 6. Get the current power consumption of the GPU after running inference.
try:
    power_after = pynvml.nvmlDeviceGetPowerUsage(handle) / 1000.0  # Power in Watts
    print(f"GPU power consumption after inference: {power_after:.2f} W")
except pynvml.NVMLError as err:
    print(f"Error getting power usage after inference: {err}")
    # Finish the task with failure if power usage cannot be obtained
    pynvml.nvmlShutdown()
    raise

# 7. Calculate the power consumed during inference (difference between after and before).
power_consumed_during_inference = power_after - power_before

# 8. Print the power consumption before and after inference, and the calculated consumption during inference.
print(f"\n--- Power Consumption Statistics ---")
print(f"Power consumption before inference: {power_before:.2f} W")
print(f"Power consumption after inference: {power_after:.2f} W")
print(f"Power consumed during inference: {power_consumed_during_inference:.2f} W")
print(f"-----------------------------------")


# 9. Shutdown pynvml.
pynvml.nvmlShutdown()
print("pynvml shutdown successfully.")

pynvml initialized successfully.
GPU device handle obtained successfully.
GPU power consumption before inference: 10.46 W
Running model inference...
Model inference completed.
GPU power consumption after inference: 29.64 W

--- Power Consumption Statistics ---
Power consumption before inference: 10.46 W
Power consumption after inference: 29.64 W
Power consumed during inference: 19.18 W
-----------------------------------
pynvml shutdown successfully.


## Report results

### Subtask:
Present the power consumption statistics.


**Reasoning**:
Print the power consumption statistics using the variables `power_before`, `power_after`, and `power_consumed_during_inference` which are already available in the kernel.



In [6]:
print(f"\n--- Power Consumption Statistics ---")
print(f"Power consumption before inference: {power_before:.2f} W")
print(f"Power consumption after inference: {power_after:.2f} W")
print(f"Power consumed during inference: {power_consumed_during_inference:.2f} W")
print(f"-----------------------------------")


--- Power Consumption Statistics ---
Power consumption before inference: 10.46 W
Power consumption after inference: 29.64 W
Power consumed during inference: 19.18 W
-----------------------------------


## Summary:

### Data Analysis Key Findings

*   The necessary libraries, `transformers` and `pynvml`, were successfully installed for running the model and measuring GPU power consumption.
*   A pretrained "gpt2" model and its tokenizer were loaded using the `transformers` library.
*   Sample text input was successfully encoded into PyTorch tensors using the loaded tokenizer.
*   GPU power consumption was successfully measured using `pynvml` before and after running the model inference.
*   Running the model inference increased the GPU power consumption from approximately 10.46 W to 29.64 W.
*   The power consumed during the inference process was calculated to be approximately 19.18 W.

### Insights or Next Steps

*   The power consumption measurement provides a baseline for evaluating the energy efficiency of different model architectures or hardware configurations for similar inference tasks.
*   Further analysis could involve measuring power consumption for different input sizes, batch sizes, or for generating longer sequences to understand the relationship between workload and energy usage.
